# 0. Upload the raw Dataset

In [1]:
import re
from utils.text_preprocessing import tokenize, stopwords
from index.prefix_tree import PrefixTree

[nltk_data] Downloading package punkt to /home/genvekt/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/genvekt/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/genvekt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/genvekt/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
from parsers.russian_parser import Russian
rus_data = Russian()
rus_data.parse_data()
len(rus_data.dataset)

No diagnosis in  74319
No diagnosis in  74308


4682

In [5]:
for rep in rus_data.dataset:
    print(rep.text)

Норма
Норма
во 2 м/р справа фокусная тень
Норма
Норма
Норма
Справа в С 1 инфильтрация л/ткани.Рек:конс. фтизиатра.
Норма
Норма
12/1 21/3,6Норма
В головке правого корня в S2 участок инфильтрации
Уч-к затен-я в Сах- С3 справа
21/1,4Норма
уч-к затен-я над д-ой справа(пневмония)
Справа в С 2 инфильтрация  л/ткани. Рек: конс. фтизиатра РКПД.
Норма
очаг тени слив хар-ра в С1-С2 с обеих сторон.
справа гол-ка корня расш-на,инфильт-на.(пневмония?)
подозрениние на очаг тени в С2 слева
Норма
Норма
Норма
мягкая очаговая тень во 2-ом межреберье слева
Норма
слева затен-е п/кар-но,очаг тени в зоне С1-2 на фоне усил сосуд рисунка
Очаговая тень в 1-м межреберье справа.
слева в С1-2 фиброз и плот очаг тени
справа очаг тени в С1,фокусная тень во2 м/реб-е.
Инфильтрация в проекции С 5 слева.
Норма
Норма
Норма
18/3Норма
Инфильтрация верхних долей обоих легких с признаками распада.
Справа в С1.2 инфильтрация л/ткани с распадом. Конс. фтизиатра.
Норма
плотная очаговая тень в S1 слева
12\6 25\6
вызов
Норма
Нор

слева в S6 инфильтрация легочной ткани
Норма
Норма
12\1
Норма
очаг тень во 2 м/реб-е справа.
Норма
Норма
Норма
справа на уровне 2 ребра очаговая тень
Норма
участок затенения в прикорн зоне справа
Очаговые тени в верхней доле слева.
Норма
плотные очаговые за тенью 3 ребра справа
Норма
Норма
уч-к фиброза с очаг тенями в 1-2 м/реб-х слева мед-но.
Норма
12\5 груглый инфильтрат в S6 слева?
Норма
Норма
Норма
Очаги в верхней доле левого легкого.
Норма
небольшой участок инфильтрации? в н/доли левого легкого
Норма
Норма
3/4
очаговая тень в в/доли справа
Спр.в верх. доле очаги.Слева во2м\р овальн.тень.
подозр на очаг тени в в/отд лёг полей
5/5
Норма
Норма
5/6 снять на макс вдохе
вызов
справа травматический плеврит
Норма
Норма
очаг тени и фибр тяжи в С1-С2 справа.
неоднородная инфильтрация легочной ткани в С3справа
Норма
плотные очаговые тени на верхушках с обеих сторон на фоне диффузного фиброза
плотная очаговая тень в S10 слева
очаговые изменения в S1-2 слева
5/4?
Норма
Норма
Слева в С1-2 очаго

In [9]:
count = 0
for report in rus_data.dataset:
    if report.label != 'норма' and 'Норма' in report.text:
        print(report.text)
        count +=1
count

Очаговая инфильтрация в проекции С 5 слева.Норма
фиброзные плотные очаги на верхушках с обеих сторонНорма
слева в заднебазальных сегментах   инфильтрация легочной тканиНорма
слева во 2 межреберье очаговые тени, справа на верхушке и поключицей калтьцинатыНорма
Слева с1+2 очаговая тень.Норма
Инфильтрация в язычковых сегмента слеваНорма
слева в средне- нижнем легочном поле затемнение с участками просветления.НормаНорма
затен-е над д-ой справа(пневмония с + динамикой)Норма
12/2,3  15,33/2  Норма
Норма
гидроторакс справа+35,15.Норма
Левый корень расширен, инфильтрирован.Норма
Однородная инфильтарция легочной ткани в С 3 вехней доли правоголегкого, отграниченная сегментарной плеврой.Норма
Остаточная инфильтрация в проекции С3 справа.Норма


14

In [11]:
from utils.text_statistics import DatasetStat
stat = DatasetStat()
stat.calculate_statistics(rus_data.dataset)

In [12]:
stat.mean_doc_length_by_label

{'норма': 1.0921501706484642, 'патология': 7.196321642429427}

# 1. Define resources
### Codes

In [13]:
codes = {
    '1':'полость',
    '2':'округлая тень',
    '3':'затемнение в легочной ткани',
    '4':'очаговые тени в легочной (более трех)',
    '5':'единичные очаговые тени в легочной ткани',
    '6':'ячеистость в легочной ткани',
    '7':'затемнение, увеличение корней (кроме фиброза)',
    '8':'фиброторакс',
    '9':'фиброз ограниченный (1-2 поля)',
    '10':'фиброз более двух легочных полей',
    '11':'фиброз одного легочного поля',
    '12':'фиброз корней выраженный',
    '13':'диффузный пневмосклероз',
    '14':'единичные фиброзные тяжи в легочной ткани',
    '15':'единичные тяжи в корнях',
    '16':'изменения в плевре (сросшееся плевра, наслоение, обызиствление)',
    '17':'междолевая "волосяная" плевра',
    '18':'деформации диафрагмы не связанные с плевральной патологией',
    '19':'состояние после операции',
    '20':'добавочная доля',
    '21':'изменение скелета грудной клетки (костные мозоли, синостозы, добавочные ребра, остеофиты и т.д.)',
    '22':'инородное тело, проецирующая на легочную ткань',
    '23':'обызиствление в легочной ткани (инодородное тело)',
    '24':'технический брак',
    '25':'прочее',
    '26':'обызиствление первичного комплекса',
    '27':'петрификаты крупные в легочной ткани',
    '28':'крупные петрификаты в корнях (диаметром не менее поперечного сечения ребра)',
    '29':'мелкие петрификаты в легочной ткани',
    '30':' множественные петрификаты (3 и более)',
    '31':'единичные петрификаты (1-2)',
    '32':'сомнительные петрификаты',
    '33':'множественные петрификаты в корнях (3 и более)',
    '34':'единичные петрификаты в корнях',
    '35':'сомнительные петрификаты в корнях',
    '36':'сердечно-сосудистая патология',
    '99':'норма'
    
}

### Vocabulary

In [14]:
voc = []
with open('resources/vocabulary.txt') as f:
    for line in f.readlines():
        voc.append(line[:len(line)-1])
print(voc)

['абсцедир', 'аксиальном', 'аксилярного', 'аорты', 'архивом', 'аталектаз', 'баз', 'без', 'ближе', 'бои', 'больше', 'больших', 'борозде', 'борозды', 'бронхопневмония', 'бронхоэктазы', 'бронхо', 'бронхо-сосудистый', 'бронхо-сосудистого', 'верхний', 'верхнедолевая', 'верхнее', 'верхне', 'верхне-долевая', 'верхние доли', 'верхняя доля', 'верхних долях', 'верхних отделах', 'верхних отделов', 'верхнее поле', 'верхних полях', 'верхней', 'верхнем', 'верхним', 'верхних', 'верхушек', 'верхушка', 'верхушках', 'верхушке', 'висцер', 'влево', 'воздух', 'воздуха', 'возможно', 'вправо', 'впроекции', 'всем', 'всю', 'выпот', 'выпуклый', 'выраженный', 'выраженно', 'гидропневмоторакс', 'гидроторакс', 'главного', 'головке', 'головка', 'гомогенного', 'грубые', 'грубых', 'грудного', 'деформация', 'деформирован', 'диафрагмы', 'диафрагмой', 'диафрагмальном', 'диафрагмального', 'динамика', 'динамики', 'динамикой', 'диссеминация', 'диссеминацией', 'дистальных', 'диффузная', 'диффузное', 'диффузный', 'добавочной'

### Russian stop words

In [15]:
stop_words = set(stopwords.words('russian'))

### Mistakes

In [16]:
mistakes = {}
with open('resources/undecided.txt', 'r') as f:
    for line in f.readlines():
        words = line.split('&')
        w1 = words[0]
        change = ''
        for i in range(1,len(words)):
            change += words[i]
        mistakes[w1[:len(w1)-1]] = change[1:len(change)-1]
mistakes

{'2ом': 'втором',
 '2ребра': 'второго ребра',
 '2стор': 'двухсторонняя',
 '5справа': '5 справа',
 '3р': 'третьего ребра',
 'аерхней': 'верхней',
 'брохопневмония': 'бронхопневмония',
 'бр/сос': 'бронхо-сосудистого',
 'верхгних': 'верхних',
 'верней': 'верхней',
 'вехней': 'верхней',
 'в-дол': 'верхнедолевая',
 'виброоза': 'xxx',
 'внрхних': 'верхних',
 'в/дол': 'верхне-долевая',
 'в/отд': 'верхнем отделе',
 'вн/дюле': 'в нижней доле',
 'во2': 'во втором',
 'во2-м': 'во втором',
 'во2м\\р': 'во втором межреберье',
 'в2-3': 'в двух или трех',
 'волн': 'волновой',
 'вплвральной': 'в плевральной',
 'впроекции': 'в проекции',
 'в1': 'в первом',
 'вс1': 'в с1',
 'вс-2': 'в с2',
 'вс9-10справа': 'в с9 и с10 справа',
 'вс3ах': 'в с3 xxx',
 'вс3справа': 'в с3 справа',
 'вс5справатень': 'в с5 справа тень',
 'вс8': 'в с8',
 'всредней': 'в средней',
 'всреднем': 'в средней',
 'выпукл': 'выпуклый',
 'выраж': 'выраженный',
 'диафр': 'диафрагмы',
 'динам': 'динамика',
 'диссем': 'диссеминация',
 'до3

In [17]:
def create_prefix_tree(data):
    pt = PrefixTree()
    for word in data:
        pt.add(word+"$")
    return pt

In [18]:
def symbol_to_end(string, symbol, cut_symbol=False):
            '''
            aaaSaaa -> aaaaaaS where S is special symbol
            '''
            index = string.find(symbol)
            return string[index + 1:] + string[:index]

In [19]:
prefix_tree = create_prefix_tree(voc)

# 2. Clean Up
### 2.1 Resolve questions

In [20]:
def resolve_questions(string):
    print(string)
    return string

In [21]:
pattern_question = re.compile(r'(?u)([\w/\\-]+)(\?)')
for report in rus_data.dataset:
    text = report.text.lower()
    #print(text)
    text = pattern_question.sub((lambda s:resolve_questions(s.group(1))),text)
    #print(text)
    #print()

пневмония
пневмония
метастазы
пневмония
слева
3/1
5/2
5
пневмония
5
5/4
справа
30
5/4
пневмония
4
диссеминация
тень
слева
саркоидоз
-киста
мтс
саркоидоз
саркоидоз
1
фиброз
30/1
21/1
саркоидоз
саркоидоз
слева
2\4
дин-ка
л-х
5/1
пневмония
5/4
отр
справа
справа
5/4
21/1
тени
5/4
5/1
ссд
3/3
рецидив
2
5/2
5/1
тени
л-х
5/4
30/4
кальцинаты
21/5
справа
фиброз
5/1
справа
2/2
21/2
5
21
пневмония
5/4
слева
5/4
пневмония
4\4
5/1
21/1
пневмония
пневмония
слева
пневмония
лгм
слева
плеврит
5/1
саркоидоз
3/2-пневмония
5/1
пневмония
инфильтрация
3/1
саркоидоз
дин-ка
пневмония
5
пневмония
саркоидоз
саркоидоз
пневмония
слева
5/3
саркоидоз
3/1
справа-инфильтрация
инфильтрация
3/2
пневмония
пневмония
слева
инфильтрации
5/4
пневмония
зоб
3
20
саркоидоз
3/3
мтс
справа
5/1
саркоидоз
ключицей
саркоидоз
вглу
тени
пневмония
пневмония
слева
саркоидоз
5
5
30
дин-ка
тень
саркоидоз
саркоидоз
5/5
5/1
л-го
5/1
сосуда


### 2.2 resolve missing space before 'норма'

In [22]:
def resolve_norm(prefix, norm):
    return prefix+" "+norm

### 2.2 Resolve word-word and word/word patterns¶

In [23]:
def resolve_reduction(before, word, after):
    # Check in mistakes
    #print(before+word+after)
    if before+word+after in mistakes:
        change = mistakes[before+word+after]
    else:
        query = after+'$'+before
        change = ''
        res = prefix_tree.find_all(query)
        if len(res)>0:
            for i in range(len(res)):
                res[i] = symbol_to_end(res[i],'$',True)
                #print(before+word+after,"->",res)
            change = res[0]
        else:
            query = before+word+after
            res = prefix_tree.find_all(query)
            if len(res)>0:
                for i in range(len(res)):
                    res[i] = symbol_to_end(res[i],'$',True)
                    #print(query,"->",res)
                change = res[0]
            else:
                shorters.add(before+word+after)
                change = before+word+after
    # print(before+word+after,"->",change)
    return change

In [24]:
def check_word(before,word, after):
    #print(word)
    if not word in voc and not word in stop_words:
        change = resolve_reduction("","",word)
    else:
        change = word
        # print(word,"->",change)
    #print("Change: ",change)
    return before+change+after

In [25]:
def fix_double_numeration(numbers, after):
    first = resolve_reduction("","",numbers[0]+"-м")
    second = resolve_reduction("","",numbers[1]+"-м")
    return first+" и "+second+after

In [26]:
def fix_numeration(number, after):
    change = resolve_reduction("","",number+"-м")
    return change+after

In [27]:
def fix_codes_loc(code, location, a,b):
    if int(code) > 0 and int(code) < 37 and code in codes:
        change = codes[code]
        a[0] += 1
        loc = resolve_reduction("","",location+"-м")
        return change+" в "+loc+" отделе"
    else:
        b[0] += 1
        return code

In [28]:
def fix_code(code, a,b):
    if int(code) > 10 and int(code) < 37 and code in codes:
        a[0] += 1
        return codes[code]
    
    else:
        b[0] += 1
        return code

In [29]:
def fix_double_c(one, two):
    return "c"+one+" и "+"c"+two

In [30]:
def fix_c(one):
    return "c"+one

In [31]:
def cals_c_s(one,two,a):
    a[0] += 1
    return one+two

In [33]:
cleared_reports = []
codes_num = [0]
numbers_num = [0]
c_num = [0]

shorters = set()
pattern_norm = re.compile(r'(?u)(\w+)(норма)')
pattern_complex = re.compile(r'(?u)(\w+)(/|\\)(\w+)(-)(\w+)')
pattern_dash = re.compile(r'(?u)(\w+)(-)(\w+)')
pattern_slash = re.compile(r'(?u)(\w+)(/|\\)(\w+)')
pattern_words = re.compile(r'(?u)(\w+)')
pattert_double_num = re.compile(r'(?u)(\d)-(\d)(\s+межреберье|\s+межреберьях)')
pattern_numeration = re.compile(r'(?u)(\d)(\s+межреберье|\s+межреберьях)')
pattern_double_c = re.compile(r'(?u)(с )(\d+),(\d+)')
pattern_c = re.compile(r'(?u)(с )(\d+)')
pattern_c_num = re.compile(r'(?u)(с|s)(\d+)')
pattern_code_loc = re.compile(r'(?u)(\d+)(\\|/)(\d+)')
pattern_code = re.compile(r'(\d+)')
for i in range(len(rus_data.dataset)):
    text = rus_data.dataset[i].text.lower()
    if text == 'норма':
        continue
    print(text)
    text = pattern_norm.sub((lambda s:resolve_norm(s.group(1),s.group(2))),text)
   # print(text)
    text = pattern_complex.sub((lambda s:check_word("",s.group(1)+s.group(2)+s.group(3)+s.group(4)+s.group(5),"")),text)
    text = pattern_dash.sub((lambda s:resolve_reduction(s.group(1),s.group(2),s.group(3))),text)
    #print(text)
    text = pattern_slash.sub((lambda s:resolve_reduction(s.group(1),s.group(2),s.group(3))),text)
    #print(text)
    text = pattern_words.sub((lambda s:check_word("",s.group(1),"")),text)
    text = pattert_double_num.sub((lambda s:fix_double_numeration([s.group(1),s.group(2)], s.group(3))),text)
    text = pattern_numeration.sub((lambda s:fix_numeration(s.group(1),s.group(2))),text)
    text = pattern_double_c.sub((lambda s:fix_double_c(s.group(2),s.group(3))),text)
    text = pattern_c.sub((lambda s:fix_c(s.group(2))),text)
    text = pattern_code_loc.sub((lambda s:fix_codes_loc(s.group(1),s.group(3),codes_num,numbers_num)),text)
    text = pattern_code.sub((lambda s:fix_code(s.group(1),codes_num,numbers_num)),text)
    text = pattern_c_num.sub((lambda s:cals_c_s(s.group(1),s.group(2),c_num)),text)
    print(text)
    print()
    rus_data.dataset[i].text = text
    
    
shorters
print(codes_num)
print(numbers_num)
print(c_num)

во 2 м/р справа фокусная тень
во втором межреберье справа фокусная тень

справа в с 1 инфильтрация л/ткани.рек:конс. фтизиатра.
справа в c1 инфильтрация легочной ткани.рекомендация:консультация. фтизиатра.

12/1 21/3,6норма
фиброз корней выраженный в первом отделе изменение скелета грудной клетки (костные мозоли, синостозы, добавочные ребра, остеофиты и т.д.) в третьем отделе,6 норма

в головке правого корня в s2 участок инфильтрации
в головке правого корня в s2 участок инфильтрации

уч-к затен-я в сах- с3 справа
участок затенения в xxx- с3 справа

21/1,4норма
изменение скелета грудной клетки (костные мозоли, синостозы, добавочные ребра, остеофиты и т.д.) в первом отделе,4 норма

уч-к затен-я над д-ой справа(пневмония)
участок затенения над дугой справа(пневмония)

справа в с 2 инфильтрация  л/ткани. рек: конс. фтизиатра ркпд.
справа в c2 инфильтрация  легочной ткани. рекомендация: консультация. фтизиатра ркпд.

очаг тени слив хар-ра в с1-с2 с обеих сторон.
очаг тени слив характера в с

7/2  3/3?
затемнение, увеличение корней (кроме фиброза) в втором отделе  затемнение в легочной ткани в третьем отделе?

очаг тени в с1-с2 справа(рецидив?)
очаг тени в с1-с2 справа(рецидив?)

округлое образование примыкающее к правому контуру тени сердца
округлое образование примыкающее к правому контуру тени сердца

участок затен слева 3 4 м/реб
участок затенен слева 3 четвертом межреберье

27/1
петрификаты крупные в легочной ткани в первом отделе

35  7  16/3  25
сомнительные петрификаты в корнях  7  изменения в плевре (сросшееся плевра, наслоение, обызиствление) в третьем отделе  прочее

участок инфильтрации в прикорневой зоне слева.
участок инфильтрации в прикорневой зоне слева.

слева в с1-2 инфильтрация л/ткани. рек: конс. фтизиатра ркпд.
слева в с1-2 инфильтрация легочной ткани. рекомендация: консультация. фтизиатра ркпд.

инфильтрация легочной ткани в нижней доли справа
инфильтрация легочной ткани в нижней доли справа

фокус тень в в/отд-х л/поля справа+21/1,2,3
фокус тень в вер

инфильтрация в c8 слева.

затен-е в с8 слева+15.(пневмония?)
затенение в с8 слева+единичные тяжи в корнях.(пневмония?)

очаг тени в зоне с1-2 с обеих сторон.
очаг тени в зоне с1-2 с обеих сторон.

фокус тень в с1 справа.
фокус тень в с1 справа.

справа в зоне с1 фиброз тяжи и очаг тени
справа в зоне с1 фиброз тяжи и очаг тени

справа в нижней доле бронхопневмония.
справа в нижней доле бронхопневмония.

корни легких уплотнены,расширены,бугристые.
корни легких уплотнены,расширены,бугристые.

35 15\15 слева за тенью сердца перивазальная инфильтрация
сомнительные петрификаты в корнях единичные тяжи в корнях в единичные тяжи в корнях-м отделе слева за тенью сердца перивазальная инфильтрация

подозр на очаг тени в 1 м/реб справа
подозрение на очаг тени в первом межреберье справа

в с1-2 верхних долей очаг тени.
в с1-2 верхних долей очаг тени.

очаг тень в 1 м/реб-е слева.
очаг тень в первом межреберье слева.

жидкость в плевральной полости справа
жидкость в плевральной полости справа

аталек

инфильтрация язычковых сегментов слева

затенение в с5 справа(пневмония)
затенение в с5 справа(пневмония)

снять в лордозе
снять в лордозе

11,29/1
фиброз одного легочного поля,мелкие петрификаты в легочной ткани в первом отделе

18,25/3норма
деформации диафрагмы не связанные с плевральной патологией,прочее в третьем отделе норма

подозр на очаг тени в 1 м/реб-х с обеих сторон
подозрение на очаг тени в первом  межреберьях с обеих сторон

очаг тени и фибр тяжи в с1 справа.
очаг тени и фиброз тяжи в с1 справа.

плеврит справа
плеврит справа

крупноочаговые тени в s1-2 справа, состояние после удаления нижней доли справа
крупноочаговые тени в s1-2 справа, состояние после удаления нижней доли справа

уч-к затен-я над д-ой справа.
участок затенения над дугой справа.

30/5.   р+.
 множественные петрификаты (3 и более) в пятом отделе.   р+.

очаги в средних легочных полях справа и слева.
очаги в средних легочных полях справа и слева.

затенение средней доли правого легкого выпот в синусе справ

In [35]:
stat = DatasetStat()
stat.calculate_statistics(rus_data.dataset)

In [36]:
stat.mean_doc_length_by_label

{'норма': 1.8071672354948805, 'патология': 8.425577416595381}

In [41]:
rus_data.dataset[2].text

'во втором межреберье справа фокусная тень'